In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



gdp_df = pd.read_csv("../data/UNdata_Export_20230915_015125007.csv")

gdp_df.head(20)

gdp_df.drop(columns="Value Footnotes")


renamed = gdp_df.rename(columns={gdp_df.columns[0]:'Country',gdp_df.columns[1]: 'Year', gdp_df.columns[2]: 'GDP_Per_Capita'})

num_columns = gdp_df.shape[1]
num_columns

num_rows = gdp_df.shape[0]
num_rows

column_types = gdp_df.dtypes
column_types


renamed = gdp_df.rename(columns={gdp_df.columns[0]:'Country',gdp_df.columns[1]: 'Year', gdp_df.columns[2]: 'GDP_Per_Capita'})

renamed.dtypes

diff_years = renamed['Year'].unique()

count_per_year = renamed['Year'].value_counts()
count_per_year

diff_countries = renamed['Country'].unique()
diff_countries

num_diff_countries = renamed['Country'].nunique()
num_diff_countries



242